In [3]:
# ============================================================
# MODEL TRAINING AND EVALUATION NOTEBOOK
# ============================================================

# STEP 1: Import Dependencies
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# ============================================================
# STEP 2: Load Configurations
# ============================================================

with open("../config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

base_path = config["data"]["base_path"]
image_size = tuple(config["data"]["image_size"])
batch_size = config["data"]["batch_size"]
epochs = 10
lr = config["model"]["learning_rate"]
dropout = config["model"]["dropout"]

print("Configuration loaded successfully.")

# ============================================================
# STEP 3: Prepare Data Generators (All in Grayscale)
# ============================================================

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(base_path, "train"),
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="binary"
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(base_path, "val"),
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="binary"
)

test_gen = test_datagen.flow_from_directory(
    os.path.join(base_path, "test"),
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="binary",
    shuffle=False
)

print("Data generators ready.")

# ============================================================
# STEP 4: Define Model Architectures
# ============================================================

def build_simple_cnn(input_shape=(180,180,1), dropout=0.3, lr=0.001):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dropout(dropout),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_vgg16_model(input_shape=(180,180,1), dropout=0.3, lr=0.001):
    # Convert grayscale to 3 channels for pretrained model
    base = VGG16(weights='imagenet', include_top=False, input_shape=(180,180,3))
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base.input, outputs=output)
    for layer in base.layers[:-4]:
        layer.trainable = False
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_efficientnet_model(input_shape=(180,180,3), dropout=0.3, lr=0.001):
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base.input, outputs=output)
    for layer in base.layers[:-5]:
        layer.trainable = False
    model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# ============================================================
# STEP 5: Train Models
# ============================================================

models_dict = {
    "SimpleCNN": build_simple_cnn(),
    "VGG16": build_vgg16_model(),
    "EfficientNetB0": build_efficientnet_model()
}

histories = {}
results = {}

for name, model in models_dict.items():
    print(f"\nTraining {name} ...")
    if name == "SimpleCNN":
        history = model.fit(train_gen, validation_data=val_gen, epochs=epochs)
    else:
        # Convert grayscale to RGB for pretrained models
        rgb_train = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
            os.path.join(base_path, "train"),
            target_size=image_size,
            batch_size=batch_size,
            color_mode="rgb",
            class_mode="binary"
        )
        rgb_val = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
            os.path.join(base_path, "val"),
            target_size=image_size,
            batch_size=batch_size,
            color_mode="rgb",
            class_mode="binary"
        )
        history = model.fit(rgb_train, validation_data=rgb_val, epochs=epochs)

    histories[name] = history
    loss, acc = model.evaluate(test_gen)
    results[name] = acc
    print(f"{name} Test Accuracy: {acc:.4f}")

# ============================================================
# STEP 6: Compare Model Performance
# ============================================================

print("\nModel Accuracy Comparison:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")

# Plot accuracy and loss curves
plt.figure(figsize=(12, 6))
for name, hist in histories.items():
    plt.plot(hist.history['accuracy'], label=f'{name} Train Acc')
    plt.plot(hist.history['val_accuracy'], label=f'{name} Val Acc')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
for name, hist in histories.items():
    plt.plot(hist.history['loss'], label=f'{name} Train Loss')
    plt.plot(hist.history['val_loss'], label=f'{name} Val Loss')
plt.title('Model Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

# ============================================================
# STEP 7: Evaluate Best Model
# ============================================================

best_model_name = max(results, key=results.get)
print(f"\nBest Model: {best_model_name}")

best_model = models_dict[best_model_name]
pred = (best_model.predict(test_gen) > 0.5).astype("int32")

print("\nClassification Report:")
print(classification_report(test_gen.classes, pred))

cm = confusion_matrix(test_gen.classes, pred)
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'{best_model_name} Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


ModuleNotFoundError: No module named 'yaml'